## Optimizing tracker parameters by Evolutionary algorithm 

In [10]:
# single gene: [t_high, t_low, t_new, t_match] - t_buff is not very important
# average time for one video process ~ 1 min 10 sec


def evolutionary_algorithm(number_of_generations=20, population_size=5, stop_count=10):
    current_generation = 0
    list_of_fitnesses = []
    min_values_list = []
    operand_type = []
    crossover_type = []
    mutation_type = []
    crossover_value = []
    mutation_value = []
    counter = 0

    parameters_population = initialize_population(population_size)
    print(parameters_population)

    for parameters_set in parameters_population:
        tracker_results = process_video(parameters_set)
        list_of_fitnesses.append(operative_function(tracker_results))
    print(list_of_fitnesses)

    current_min_value = min(list_of_fitnesses)
    best_individual_idx = list_of_fitnesses.index(current_min_value)
    best_individual = parameters_population[best_individual_idx]
    min_values_list.append(current_min_value)

    while current_generation <= number_of_generations:
        # new_population = []
        # fitness_table_for_current_population = []

        # selecting
        print(len(parameters_population), len(list_of_fitnesses))
        selected_population, new_list_of_fitnesses = select_population(parameters_population=parameters_population, list_of_fitnesses=list_of_fitnesses)
        fitness_table_for_current_population = new_list_of_fitnesses
        new_population = selected_population
        print(selected_population)
        print(len(fitness_table_for_current_population))
        while True:
            # genetic_operation = random.choices(list(Operations), weights=[0.2, 0.8])
            # genetic_operation = genetic_operation[0]
            genetic_operation = 'cross'

            if genetic_operation == 'cross':
                
                par1_idx = np.random.randint(0, len(selected_population))
                # print(par1_idx)
                # par2_idx = np.random.randint(0, len(selected_population)-1)
                cross_type = 'simple'
                # cross_type = random.choices(list(Crossover))[0]
                operand_type.append('Crossover')

                # children_1 = selected_population[par1_idx]
                # children_2 = selected_population[par2_idx]
                if cross_type == 'simple':
                    children_1, children_2 = new_children(selected_population[par1_idx], fitness_table_for_current_population[par1_idx])
                    # print(children_1, children_2)
                    children_1_operative = operative_function(process_video(children_1))
                    # print(f"ch_1 op: {children_1_operative}")
                    if len(new_population) + 2 <= len(parameters_population):
                        children_2_operative = operative_function(process_video(children_2))
                        if children_2_operative > children_1_operative:
                            children_1, children_2 = children_2, children_1
                            children_1_operative, children_2_operative = children_2_operative, children_1_operative
                    crossover_type.append('simple')
                    mutation_type.append(np.NaN)

                new_population.append(children_1)
                fitness_table_for_current_population.append(children_1_operative)
                print(len(fitness_table_for_current_population))
                if len(new_population) != len(parameters_population):
                    new_population.append(children_2)
                    fitness_table_for_current_population.append(children_2_operative)
                    print(len(fitness_table_for_current_population))
                print(f"new population: {new_population}")
                
            if len(new_population) == len(parameters_population):
                break

        max_selected = min(fitness_table_for_current_population)
        if max_selected >= current_min_value:
            counter += 1
        if max_selected < current_min_value:
            counter = 0
            current_min_value = max_selected
            idx_selected = fitness_table_for_current_population.index(current_min_value)
            best_individual = selected_population[idx_selected]
        if counter == stop_count:
            break
        parameters_population = new_population
        list_of_fitnesses = fitness_table_for_current_population
        current_generation += 1
        print(current_generation)
        min_values_list.append(current_min_value)
    return best_individual, current_min_value, min_values_list, operand_type, crossover_type, mutation_type


best_individual, current_min_value, min_values_list, operand_type, crossover_type, mutation_type = evolutionary_algorithm()
print(best_individual)

[{'track_high_thresh': 0.5, 'track_low_thresh': 0.1, 'new_track_thresh': 0.6, 'match_thresh': 0.8}, {'track_high_thresh': 0.1293, 'track_low_thresh': 0.2637, 'new_track_thresh': 0.8331, 'match_thresh': 0.3709}, {'track_high_thresh': 0.7771, 'track_low_thresh': 0.2907, 'new_track_thresh': 0.9422, 'match_thresh': 0.1684}, {'track_high_thresh': 0.4911, 'track_low_thresh': 0.8134, 'new_track_thresh': 0.907, 'match_thresh': 0.4019}, {'track_high_thresh': 0.4778, 'track_low_thresh': 0.005, 'new_track_thresh': 0.3896, 'match_thresh': 0.7954}]
[0.004333519722491661, 1.4435489574117764, 9.485565044196065, 9.17033803893478, 0.13172616143131]
5 5
[{'track_high_thresh': 0.5, 'track_low_thresh': 0.1, 'new_track_thresh': 0.6, 'match_thresh': 0.8}, {'track_high_thresh': 0.4778, 'track_low_thresh': 0.005, 'new_track_thresh': 0.3896, 'match_thresh': 0.7954}, {'track_high_thresh': 0.1293, 'track_low_thresh': 0.2637, 'new_track_thresh': 0.8331, 'match_thresh': 0.3709}]
2


IndexError: list index out of range

In [358]:
np.random.randint(0, 1)

0

In [2]:
import numpy as np
import sys
import os
import ruamel.yaml
import random
from ruamel.yaml import YAML
from copy import copy
from enum import Enum


def new_children(parent:dict, parent_operative):
    names = ["track_high_thresh", "track_low_thresh", "new_track_thresh", "match_thresh"]
    add_number = np.random.randint(0, 4)
    chosen_names = random.sample(names, add_number)
    # print(chosen_names)
    percentage = np.log(parent_operative + 1)/100 + 1
    print(f"percentage: {percentage}")
    children_1 = {}
    children_2 = {}
    for key, val in parent.items():
        if key in chosen_names:
            children_1[key] = round(float(val*percentage if val*percentage < 1 else 1), 4)
            children_2[key] = round(float(val/percentage), 4)
        else:
            children_1[key] = round(float(val/percentage), 4)
            children_2[key] = round(float(val*percentage if val*percentage < 1 else 1), 4)
    
    return children_1, children_2

In [3]:
import numpy as np
import sys
import os
import ruamel.yaml
import random
import math
from ruamel.yaml import YAML
from copy import copy
from enum import Enum


def select_population(parameters_population, list_of_fitnesses, selection_size=60):
    selected_population = []
    selected_fitnesses = []
    list_of_weights = [1/i for i in list_of_fitnesses]
    selection_size = int(len(parameters_population)*selection_size/100)

    copy_population = copy(parameters_population)

    for i in range(selection_size):
        selected_individual = random.choices(copy_population, weights=list_of_weights, k=1)

        selected_population.append(selected_individual[0])
        idx_selected = copy_population.index(selected_individual[0])
        selected_fitnesses.append(list_of_fitnesses[idx_selected])

        copy_population.remove(selected_individual[0])
        list_of_fitnesses.pop(idx_selected)
        list_of_weights.pop(idx_selected)

    return selected_population, list_of_fitnesses


def initialize_population(population_num):
    parameters_names = ["track_high_thresh", "track_low_thresh", "new_track_thresh", "match_thresh"]
    parameters_population = [{"track_high_thresh":0.5, "track_low_thresh":0.1, "new_track_thresh":0.6, "match_thresh":0.8}]
    for i in range(population_num-1):
        parameters_set = {}
        for name in parameters_names:
            parameters_set[name] = round(random.random(), 4)
        parameters_population.append(parameters_set)
    return parameters_population


def operative_function(params:dict):
    ID_COUNT = 100
    TOTAL_LIFE = 2200
    AVG_LIFE = 22.4
    MAX_ID_COUNT = 400
    result = (abs(params["id_max"]-MAX_ID_COUNT)/MAX_ID_COUNT) * (1 + (abs(params["id_count"]-ID_COUNT)/ID_COUNT)) * \
             (1 + (abs(params["total_life"]-TOTAL_LIFE)/TOTAL_LIFE)) *\
             (1 + (abs(params["avg_life"]-AVG_LIFE)/AVG_LIFE)) * (1/params["avg_iou"])
    if math.isinf(result):
        result = 1000
    return result
    

def process_video(parameters_set:dict):
    yaml = YAML()
    with open('../trackers/tracker.yaml', 'r') as fp:
        data = yaml.load(fp)
    data["track_high_thresh"] = parameters_set["track_high_thresh"]
    data['track_low_thresh'] = parameters_set["track_low_thresh"]
    data['new_track_thresh'] = parameters_set["new_track_thresh"]
    data['match_thresh'] = parameters_set["match_thresh"]
    # print(data)

    with open('../trackers/tracker.yaml', 'w') as file:
        yaml.dump(data, file)

        track_history, iou_values = get_tracker_results()

        if len(track_history) >= 1:
            id_max = max(track_history.keys())
            id_count = len(track_history.keys())
            tot_life = 0
            for el in track_history.values():
                tot_life += len(el)
            avg_life = tot_life/id_count
            avg_iou = np.nanmean(iou_values)
        else:
            id_max = np.inf
            id_count = np.finfo(float).eps
            tot_life = np.finfo(float).eps
            avg_life = np.finfo(float).eps
            avg_iou = np.finfo(float).eps

    return {'id_max':id_max, 'id_count':id_count, 'total_life':tot_life, 'avg_life':avg_life, 'avg_iou':avg_iou}
    

# pop = initialize_population(4)
# print(pop)

# sample_indicators = {'id_max':200, 'id_count':110, 'total_life':1700, 'avg_life':27, 'avg_iou':0.73}
# sample_indicators_1 = {'id_max':100, 'id_count':100, 'total_life':2200, 'avg_life':20, 'avg_iou':0.79}

# print(operative_function(sample_indicators))
# print(operative_function(sample_indicators_1))

# pop = [[0.1, 0.1, 0.1, 0.1], [0.2, 0.2, 0.2, 0.2], [0.3, 0.3, 0.3, 0.4], [0.5, 0.5, 0.5, 0.5], [0.7, 0.7, 0.7, 0.7]]
# fit = [143, 193, 243, 50, 100]

# print(select_population(parameters_population=pop, list_of_fitnesses=fit))

In [4]:
from enum import Enum
import numpy as np
from random import sample, choices
from copy import copy


def swap_mutation(solution):
    """
    Mutation function , swapping to random gens in individual
    """
    gen1 = np.random.randint(0, len(solution) - 1)
    gen2 = np.random.randint(0, len(solution) - 1)
    if gen1 != gen2:
        solution[gen1], solution[gen2] = solution[gen2], solution[gen1]
    else:
        pass
    return solution


def inversion_mutation(solution):
    """
    Inversion mutation function ,inverting gens in random section
    """
    gens = sample(range(0, len(solution)), 2)
    if gens[0] > gens[1]:
        gens[0], gens[1] = gens[1], gens[0]
    solution[gens[1]:gens[0]+1] = solution[gens[1]:gens[0]+1][::-1]
    return solution


def scramble_mutation(solution):
    """
    Scramble mutation function ,randomly swapping gens in random section
    """
    gens = sample(range(0, len(solution)), 2)
    if gens[0] > gens[1]:
        gens[0], gens[1] = gens[1], gens[0]
    scramble_tab = solution[gens[0]:gens[1]+1]
    new_queue = sample(scramble_tab, len(scramble_tab))
    solution[gens[0]:gens[1]+1] = new_queue
    return solution


class Mutations(Enum):
    swap = 1
    inversion = 2
    scramble = 3


class Operations(Enum):
    mutation = 1
    crossover = 2


class Crossover(Enum):
    pmx = 1
    cx = 2
    ox = 3


def pmx(parent_1: list, parent_2: list):
    return 0

def cx(parent_1: list, parent_2: list):
    return 0

def ox(parent_1: list, parent_2: list):
    return 0

In [5]:
import cv2
import math
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from natsort import natsorted
from ouster import client
from ouster import pcap
from ouster.client import Scans, XYZLut, SensorInfo, destagger
from ouster.client.data import LidarScan
from ultralytics import YOLO
from contextlib import closing
from collections import defaultdict

def get_xyxy_from_txt(label):
    gt_boxes = list(map(float, label.split(' ')[1:]))
    x_center = gt_boxes[0]*1024
    y_center = gt_boxes[1]*128
    box_width = gt_boxes[2]*1024
    box_height = gt_boxes[3]*128
    
    x1 = int(x_center - (box_width / 2))
    y1 = int(y_center - (box_height / 2))
    x2 = int(x_center + (box_width / 2))
    y2 = int(y_center + (box_height / 2))

    return [x1, y1, x2, y2]


def calculate_iou(box1, box2):
    x1_box1, y1_box1, x2_box1, y2_box1 = box1
    x1_box2, y1_box2, x2_box2, y2_box2 = box2

    x_left = max(x1_box1, x1_box2)
    y_top = max(y1_box1, y1_box2)
    x_right = min(x2_box1, x2_box2)
    y_bottom = min(y2_box1, y2_box2)
    
    intersection_area = max(0, x_right - x_left) * max(0, y_bottom - y_top)
    
    area_box1 = (x2_box1 - x1_box1) * (y2_box1 - y1_box1)
    area_box2 = (x2_box2 - x1_box2) * (y2_box2 - y1_box2)
    
    union_area = area_box1 + area_box2 - intersection_area
    
    iou = intersection_area / union_area if union_area > 0 else 0
    return iou


def associate_boxes(boxes1, boxes2):
    associations = []
    for box1 in boxes1:
        max_iou = 0
        associated_box = None
        for box2 in boxes2:
            iou = calculate_iou(box1, box2)
            if iou > max_iou:
                max_iou = iou
                associated_box = box2
        associations.append((box1, associated_box, max_iou))
    return associations


def get_tracker_results():

    output_dict = {3:["GO AHEAD", (0, 255, 0)], 2:["BE CAREFUL", (0, 155, 100)], 1:["SLOW DOWN", (0, 100, 155)], 
                    0:["BREAK", (0, 0, 255)]}

    # Load the YOLOv8 model
    model = YOLO('../weights/best_3000_n_200.pt')

    # Paths to pcap and json files
    metadata_path = "C:/Users/szyme/Ouster/data/PKR_test1/test4.json"
    pcap_path = "C:/Users/szyme/Ouster/data/PKR_test1/test4.pcap"

    # Making PacketSource from data
    with open(metadata_path, 'r') as f:
        metadata = client.SensorInfo(f.read())

    fps = int(str(metadata.mode)[-2:])
    width = int(str(metadata.mode)[:4])
    height = int(str(metadata.prod_line)[5:])

    pcap_file = pcap.Pcap(pcap_path, metadata)

    xyz_lut = client.XYZLut(metadata) #call cartesian lookup table

    # Store the track history
    track_history = defaultdict(lambda: [])     # dictionary: {key=id, value=[xyz_val_0, ... ,xyz_val_99]}
    box_history = defaultdict(lambda: [])
    # iou_values = []
    iou_values = np.full(1, np.nan)

    directory_path = 'groundtruthing_labels/'

    with closing(Scans(pcap_file)) as scans:

        save_path = "C:/Users/szyme/Ouster/Dangerous-situations-with-pedastrians/results_mp4"
        vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
        files = os.listdir(directory_path)
        files_sorted = natsorted(files)

        i = 0
        for scan, filename in zip(scans, files_sorted):
            i += 1

            f = os.path.join(directory_path, filename)
            if os.path.isfile(f):
                labels_file = f
            with open(labels_file, 'r') as file:
                labels = file.readlines()

            sig_field = scan.field(client.ChanField.SIGNAL)
            sig_destaggered = destagger(metadata, sig_field)
            scaling_factor = 0.004
            scaled_arr = sig_destaggered / (0.5 + scaling_factor * sig_destaggered)
            signal_image = scaled_arr.astype(np.uint8)
            combined_img = np.dstack((signal_image, signal_image, signal_image))

            xyz_destaggered = client.destagger(metadata, xyz_lut(scan))

            results = model.track(source=combined_img, persist=True, imgsz=1024, tracker='../trackers/tracker.yaml', verbose=False)

            boxes = results[0].boxes.xyxy.cpu().numpy().astype(int)

            if (results[0].boxes.id == None):
                ids = ''
            else:
                ids = results[0].boxes.id.cpu().numpy().astype(int)

            priority = 3
            distance = 0

            gt_boxes = []
            for label in labels:
                gt_boxes.append(get_xyxy_from_txt(label))

            associations = associate_boxes(gt_boxes, boxes)
            for box1, box2, iou in associations:
                if isinstance(box2, np.ndarray):
                    iou_values = np.append(iou_values, iou)
                    # iou_values.append(iou)
                else:
                    iou_values = np.append(iou_values, np.nan)

            for box, id in zip(boxes, ids):

                center_x, center_y = int((box[0] + box[2])/2), int((box[1] + box[3])/2) #

                xyz_val = xyz_destaggered[(center_y, center_x)] #get the (x,y,z) coordinates with the lookup table 

                track = track_history[id] #save the (x,y,z) coordinates for distance calculation
                track.append(xyz_val)

                cv2.rectangle(combined_img, (box[0], box[1]), (box[2], box[3]), output_dict[priority][1], 2)
                cv2.rectangle(combined_img, (box[0], box[1]+2), (box[0]+20+len(str(id))*11, box[1]-12), (255, 255, 255), -1)
                cv2.putText(combined_img, f"Id {id}", (box[0], box[1]), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

            cv2.putText(combined_img, f"{output_dict[priority][0]}", (470, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, output_dict[priority][1], 2)

            if True:
                cv2.imshow("YOLOv8 Tracking", combined_img)
                cv2.waitKey(1)  # 1 millisecond

            vid_writer.write(combined_img)

            if cv2.waitKey(1) & 0xFF == ord("q"):
                break
            if i >= 1000:
                break

        vid_writer.release()
        cv2.destroyAllWindows()
    return track_history, np.array(iou_values)